# Test PointPillars on ARCS data

In [28]:
# Libraries
import bbox
import itertools
import math
import os
import sys
import numpy as np
import pandas as pd
from contextlib import redirect_stdout
from mmdet3d.apis import LidarDet3DInferencer
from pathlib import Path

In [29]:
# Initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/rachel/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/rachel/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [30]:
# Directory paths
ARCS_LABELS = '../data/eval_data/labels'
ARCS_DATA_DIR = '../data/eval_data/arcs'
ARCS_FILTERED_DATA_DIR = '../data/eval_data/arcs_azimuth_filtered'
ARCS_LABEL_FILTERED_DATA_DIR = '../data/eval_data/arcs_label_filtered'

In [31]:
# The 3D bbox object requires quaternion values. This function converts the yaw to these values
def yaw_to_quaternion(yaw):
    """
    Converts a yaw angle (rotation about the z-axis) to quaternion coordinates.
    
    Parameters:
    - yaw (float): The yaw angle in radians.
    
    Returns:
    - tuple of (rw, rx, ry, rz): The quaternion representation of the yaw.
    """
    # Compute the components of the quaternion
    rw = math.cos(yaw / 2.0)
    rz = math.sin(yaw / 2.0)
    
    # rx and ry are zero because the rotation is only about the z-axis
    return (rw, 0, 0, rz)

In [32]:
# This function takes x, y, z, dx, dy, dz, yaw and returns a 3D bbox object from the bbox package
# Yaw is in radians
def get3DBbox(x, y, z, dx, dy, dz, yaw):
    # Example usage:
    rw, rx, ry, rz = yaw_to_quaternion(yaw)
    bbox_obj = bbox.BBox3D(x, y, z, length=dx, width=dy, height=dz, rw=rw, rx=rx, ry=ry, rz=rz)
    return bbox_obj

In [33]:
def calculate_iou(bbox1, bbox2):
    bbox_obj1 = get3DBbox(*bbox1)
    bbox_obj2 = get3DBbox(*bbox2)
    return bbox.metrics.jaccard_index_3d(bbox_obj1, bbox_obj2)

In [34]:
# _evaluate frame takes a prediction dictionary output by the model, and a list of ground truths
# from the label file, and returns the TPs, FPs, and FNs for one lidar frame
def _evaluate_frame(predictions, ground_truths, threshold=0.25):
    TPs, FPs, FNs = 0, 0, len(ground_truths)
    used_gt = set()
    
    for pred in predictions['predictions']:
        for label, score, bbox in zip(pred['labels_3d'], pred['scores_3d'], pred['bboxes_3d']):
            if score < threshold:
                continue

            best_iou = 0
            best_gt = None
            for gt in ground_truths:
                iou = calculate_iou(bbox, gt[8:])
                if iou > best_iou:
                    best_iou = iou
                    best_gt = tuple(gt)

            if best_iou > threshold:  # Example IoU threshold for a match
                if best_gt not in used_gt:
                    used_gt.add(best_gt)
                    TPs += 1
                    FNs -= 1
                else:
                    FPs += 1
            else:
                FPs += 1
    return TPs, FPs, FNs

In [35]:
def parse_ground_truths(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = []
            # Add the category as a string
            bbox.append(parts[0])
            # Extract the bounding box dimensions and location as 
            bbox = bbox + [float(value) for value in parts[1:15]]  
            bbox[2] = int(bbox[2])
            labels.append(bbox)
    return labels

In [36]:
generated_problem_file_list = []

In [37]:
# _evaluate frame takes a lidar and label file path and
# and returns the TPs, FPs, and FNs for one lidar frame
def evaluate_frame(lidar_file_path, label_file_path):
    inputs = dict(points=str(lidar_file_path))
    
    # Get predictions
    try:
        predictions = inferencer(inputs)
            # Get ground truths
        ground_truths = parse_ground_truths(label_file_path)

        TPs, FPs, FNs = _evaluate_frame(predictions, ground_truths)
    except:
        # Add file id to the problem file list
        print(str(lidar_file_path))
        lidar_filename = os.path.basename(lidar_file_path)
        file_id, extension = os.path.splitext(lidar_filename)
        generated_problem_file_list.append(file_id)
        TPs, FPs, FNs = 0, 0, 0
    
    return TPs, FPs, FNs

In [38]:
# Problem files. I'll find out why later
problem_file_ids = ['000522', '003066', '000556', '004340', '001016', '004224', '005318', '004344', '001014', '001027', 
                    '006440', '001026', '000869', '001031', '000530', '004928', '000765', '004926', '003395', '000864', 
                    '004226', '000550', '006437', '000538', '004930', '000527', '004223', '000766', '001012', '005320', 
                    '006438', '004014', '001010', '001013', '003439', '004016', '001008', '000866', '000662', '001015', 
                    '000523', '001018', '000554', '000524', '000528', '006445', '005156', '004347', '000539', '000533', 
                    '004225', '004013', '004227', '000558', '003436', '004348', '000525', '004345', '004015', '005319', 
                    '004012', '005321', '003397', '000541', '000534', '006436', '000549', '000553', '000767', '001023', 
                    '006442', '000865', '003065', '004346', '003398', '004230', '000764', '000665', '000520', '003067', 
                    '006444', '001011', '000763', '005157', '001025', '000548', '005158', '000543', '001019', '001024', 
                    '001033', '005155', '000868', '004288', '000542', '003440', '000529', '006443', '000762', '000535', 
                    '004342', '000540', '001032', '000552', '004011', '004251', '004929', '004054', '001021', '000545', 
                    '004228', '000443', '001030', '000444', '000544', '001022', '001034', '006439', '001028', '003437', 
                    '004252', '006441', '003399', '005317', '001017', '000863', '003144', '000663', '001009', '005322', 
                    '000526', '000532', '003064', '000768', '005154', '003063', '004289', '000557', '004250', '000551', 
                    '004253', '004927', '000536', '000521', '000445', '004229', '000555', '000867', '000446', '004287', 
                    '004291', '000546', '004290', '000537', '003396', '004343', '000531', '001029', '000664', '003438', 
                    '001020', '000547', '000559', '004341']
okay_file_ids = ['001731', '005977', '005368', '005924', '004402']

In [39]:
def print_summary(list_file_ids):
    for id in list_file_ids:
        print(f'\n_____________________________________________________________')
        
        file_path = f'{ARCS_LABEL_FILTERED_DATA_DIR}/{id}.bin'
        label_path = f'{ARCS_LABELS}/{id}.txt' 

        # Open point cloud file
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        df_points = pd.DataFrame(points, columns=['x', 'y', 'z', 'intensity'])

        # Print the DataFrame summary
        print(f"Summary for file ID {id}:")
        print('length: ' + str(len(df_points)))
        print("DataFrame Info:")
        df_points.info()
        print("\nDataFrame Description:")
        print(df_points.describe())
        print(f'\nlabels:')
        # Print labels
        with open(label_path, 'r') as file:
            for line in file:
                print(line)

In [40]:
# print_summary(problem_file_ids)

In [41]:
# print_summary(okay_file_ids)

In [42]:
def evaluate_dataset(dataset_path):
    label_dir = Path(ARCS_LABELS)
    dataset_dir = Path(dataset_path)
    
    total_TPs, total_FPs, total_FNs = 0, 0, 0

    for bin_file in itertools.islice(dataset_dir.iterdir(), 10):
#     for bin_file in dataset_dir.iterdir():
        if str(bin_file).endswith('.bin'):
            print('.', end='')

            lidar_filename = os.path.basename(bin_file)
            # Split the filename from the extension ('006428', '.txt')
            file_id, extension = os.path.splitext(lidar_filename)
            if file_id not in problem_file_ids:
                label_filename = file_id + '.txt'

                # Make file paths
                lidar_file_path = Path(dataset_dir, lidar_filename)
                label_file_path = Path(label_dir, label_filename)
                
                print('evaluating: ' + str(lidar_file_path))
                
                TPs, FPs, FNs = evaluate_frame(lidar_file_path, label_file_path)

                total_TPs += TPs
                total_FPs += FPs
                total_FNs += FNs
            else:
                print('skipping ' + file_id)
            
    return total_TPs, total_FPs, total_FNs

In [43]:
def test_pointpillars(dataset_path, dataset_name):
    print('Evaluating dataset: ' + dataset_path)
    # Get all TP, FP, and FN in the dataset
    TPs, FPs, FNs = evaluate_dataset(dataset_path)
    # Get metrics
    precision = TPs / (TPs + FPs) if TPs + FPs > 0 else 0
    recall = TPs / (TPs + FNs) if TPs + FNs > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Placeholder for average precision calculation
    average_precision = 0  # This will need actual precision-recall curve data

    # Organize results into a dictionary
    results = {
        'Dataset': dataset_name,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Average Precision': average_precision
    }
    print(results)

    return results

In [44]:
%%capture
# Run tests on ARCS
results = test_pointpillars(ARCS_DATA_DIR, 'ARCS')

In [45]:
# print('Metrics for ARCS')
# print('Precision: ' + str(precision))
# print('Recall: ' + str(recall))

In [46]:
%%capture
# Run tests on ARCS filtered
filter_results = test_pointpillars(ARCS_FILTERED_DATA_DIR, 'Filtered ARCS')

In [47]:
# print('Metrics for Filtered ARCS')
# print('Precision: ' + str(filter_precision))
# print('Recall: ' + str(filter_recall))

In [48]:
%%capture
# Run tests on ARCS label filtered
label_filter_results = test_pointpillars(ARCS_LABEL_FILTERED_DATA_DIR, 'Label Filtered ARCS')

In [49]:
# print('Metrics for Label Filtered ARCS')
# print('Precision: ' + str(label_filter_precision))
# print('Recall: ' + str(label_filter_recall))

In [50]:
print(generated_problem_file_list)

[]


## Show results

In [51]:
print(results)
print(filter_results)
print(label_filter_results)

{'Dataset': 'ARCS', 'Precision': 0.2702702702702703, 'Recall': 0.7407407407407407, 'F1 Score': 0.396039603960396, 'Average Precision': 0}
{'Dataset': 'Filtered ARCS', 'Precision': 0.7692307692307693, 'Recall': 0.37037037037037035, 'F1 Score': 0.5, 'Average Precision': 0}
{'Dataset': 'Label Filtered ARCS', 'Precision': 0.95, 'Recall': 0.7037037037037037, 'F1 Score': 0.8085106382978724, 'Average Precision': 0}


In [55]:
# Create DataFrame
results_df = pd.DataFrame([results, filter_results, label_filter_results])
display(results_df.hide_index())

AttributeError: 'DataFrame' object has no attribute 'hide_index'

In [54]:
print(results_df.to_string(index=False))

            Dataset  Precision   Recall  F1 Score  Average Precision
               ARCS   0.270270 0.740741  0.396040                  0
      Filtered ARCS   0.769231 0.370370  0.500000                  0
Label Filtered ARCS   0.950000 0.703704  0.808511                  0
